In [1]:
%gui qt5
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import time
import importlib
from pathlib import Path
from datetime import datetime

from numba import jit
import numpy as np
import pandas as pd

from neuropy import core

from pyphocorehelpers.preferences_helpers import set_pho_preferences
set_pho_preferences() # Set preferences
# from pyphoplacecellanalysis.General.SpikesDataframeWindow import SpikesDataframeWindow, SpikesWindowOwningMixin
# from pyphoplacecellanalysis.General.DataSeriesToSpatial import DataSeriesToSpatial
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster_Vedo import Spike3DRaster_Vedo


# curr_epoch_name = 'maze1'
# curr_epoch = curr_active_pipeline.filtered_epochs[curr_epoch_name] # <NamedTimerange: {'name': 'maze1', 'start_end_times': array([  22.26      , 1739.15336412])};>
# curr_sess = curr_active_pipeline.filtered_sessions[curr_epoch_name]
# curr_spikes_df = curr_sess.spikes_df

from scipy.sparse import coo_matrix, csr_matrix
from neuropy.utils.mixins.binning_helpers import get_bin_centers, get_bin_edges
from typing import Dict, List, Optional, Union, Tuple
from numpy.typing import NDArray
# from mayavi import mlab
# import mlab
from mayavi import mlab

from neuropy.utils.mixins.binning_helpers import get_bin_centers, get_bin_edges

# def trap_exc_during_debug(*args):
#     # when app raises uncaught exception, print info
#     print(args)


# # install exception hook: without this, uncaught exception would cause application to exit
# sys.excepthook = trap_exc_during_debug
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import create_sparse_matrix_from_bins


""" 2025-02-21 Notebook for testing `mayavi` or `mlab`
http://docs.enthought.com/mayavi/mayavi/mlab.html

"""
from mayavi import mlab
mlab.options.backend = 'envisage'
f = mlab.figure()


********************************************************************************
         to build the TVTK classes (9.4). This may cause problems.
         Please rebuild TVTK.
********************************************************************************



ERROR|2025-02-21 14:00:02,875|error creating view control [envisage.plugins.python_shell_view]
Traceback (most recent call last):
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/pyface/ui/qt/workbench/workbench_window_layout.py", line 106, in add_view
    self._qt4_add_view(view, position, relative_to, size)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/pyface/ui/qt/workbench/workbench_window_layout.py", line 469, in _qt4_add_view
    dw = self._qt4_create_view_dock_widget(view, size)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/pyface/ui/qt/workbench/workbench_window_layout.py", line 541, in _qt4_create_view_dock_widget
    view.control = view.create_control(dw.widget())
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/envisage/plugins/python_shell/view/python_shell_view.py", line 122, in create_control
    self.bind(name, value)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-pack

In [ ]:
def visualize_sparse_matrix_3d(sparse_matrix: csr_matrix, bin_centers_dict: Dict[str, NDArray]):
    """Visualizes a 3D sparse matrix using Mayavi."""

    # Convert to COO format for easier access to coordinates
    sparse_matrix_coo = sparse_matrix.tocoo()

    # Extract coordinates and values
    x = sparse_matrix_coo.row
    y_and_z = sparse_matrix_coo.col  # Combined y and z indices
    values = sparse_matrix_coo.data

    # Separate y and z coordinates (reverse the flattening)
    num_dimensions = len(bin_centers_dict)
    col_names = list(bin_centers_dict.keys())
    
    y = np.zeros(len(y_and_z), dtype=int)
    z = np.zeros(len(y_and_z), dtype=int)

    multiplier = 1
    temp_y_and_z = y_and_z.copy()
    for i in range(num_dimensions - 1, 0, -1): # Iterate backwards
        z_temp = temp_y_and_z // multiplier
        if i == (num_dimensions - 1):
            z = z_temp
        else:
            y += z_temp * multiplier
        
        temp_y_and_z = temp_y_and_z - (z_temp * multiplier)
        multiplier *= len(bin_centers_dict[col_names[i]])
    
    # Create the Mayavi figure
    mlab.figure(bgcolor=(1, 1, 1), size=(800, 600))

    # Use points3d for a scatter plot of non-zero elements
    pts = mlab.points3d(x, y, z, values, colormap="viridis", scale_factor=0.8)

    # Add axes and labels
    mlab.axes(xlabel=col_names[0], ylabel=col_names[1], zlabel=col_names[2])
    mlab.outline(pts)
    mlab.title("3D Sparse Matrix Visualization", size=0.3)

    # Show the plot
    mlab.show()


# Example Usage (3D)
data_3d = {'x': np.random.rand(500) * 10,
           'y': np.random.rand(500) * 5,
           'z': np.random.rand(500) * 2}
df_3d = pd.DataFrame(data_3d)
bin_centers_x = np.linspace(0, 10, 20)
bin_centers_y = np.linspace(0, 5, 10)
bin_centers_z = np.linspace(0, 2, 5)
bin_centers_dict_3d = {'x': bin_centers_x, 'y': bin_centers_y, 'z': bin_centers_z}
sparse_matrix_3d = create_sparse_matrix_from_bins(df_3d, bin_centers_dict_3d)

if sparse_matrix_3d is not None:
    visualize_sparse_matrix_3d(sparse_matrix_3d, bin_centers_dict_3d)